In [37]:
import time

from IPython.display import Markdown, display  # Import
from openai import OpenAI
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv
import asyncio
import pandas as pd
from io import StringIO
from src.business_requirements import *
from src.data_processing import *
from src.prompts import *
from src.simplified_business_requirements import *
from src.detailed_business_requirements import *
from src.permission_business_requirements import *
from src.prohibitions_business_requirements import *
from src.concise_summary import *
from src.table_format import *
from src.key_terms import *
warnings.filterwarnings("ignore")
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


## Define Query Function

In [38]:
async def simplified_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        simplified_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Simplified_Business_Requirements"] = results
    return df

async def detailed_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        detailed_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Detailed_Business_Requirements"] = results
    return df

async def permissions_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        permissions_buss_req(
            original_question=query,
            business_requirement=row["Business_Requirements"],
            detailed_business_requirement=row["Detailed_Business_Requirements"]
        )
        for _, row in df.iterrows()
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the generated permissions back into the DataFrame
    df["Permission_Business_Requirements"] = results
    return df

async def prohibitions_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        prohibitions_buss_req(
            original_question=query,
            business_requirement=row["Business_Requirements"],
            detailed_business_requirement=row["Detailed_Business_Requirements"]
        )
        for _, row in df.iterrows()
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the generated permissions back into the DataFrame
    df["Prohibitions_Business_Requirements"] = results
    return df

async def create_table(query:str, output_path:str, top_k:int=25):
    # Identify Requirements
    response = verbatim_business_requirements(query, top_k=top_k)

    # Convert Requirements into a table
    df = convert_str_to_df(response)
    df = df.dropna(axis=1, how='all')
    df = clean_dataframe(df, df.columns[0])
    df["Business_Requirements"] = df[df.columns[0]]
    df = df.drop(df.columns[0], axis=1)

    # Identified Simplified Business Requirements
    df = await simplified_business_requirement(query=query, df=df)

    # Identified Detailed Business Requirements
    df = await detailed_business_requirement(query=query, df=df)

    # Identified Detailed Business Permissions
    df = await permissions_business_requirement(query=query, df=df)

    # Identified Detailed Prohibitions
    df = await prohibitions_business_requirement(query=query, df=df)

    # Save File
    df.to_excel(output_path, index=False)

    # Concise Summary
    summary = await concise_summary(query)

    # Table Format
    table_markdown = await table_to_markdown(df.to_json())

    # Key Terms
    response = " ".join([regulation_summary, table_markdown])
    key_term_markdown, full_context = await key_terms(original_question=query, response=response, top_k=top_k)

    return df, summary, table_markdown, key_term_markdown, full_context

query = "What are the business requirements for Regulation K?"
output_path = "../Data/Results/Regulation_K.xlsx"
# Run the async function using asyncio.run
df, regulation_summary, table_markdown, key_terms_markdown, full_context = await create_table(query, output_path)
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements,Permission_Business_Requirements,Prohibitions_Business_Requirements
0,Business RequirementBanking institutions shall...,The business requirement for Regulation K stat...,"I'm sorry, but I cannot provide detailed requi...",## **Permissions Based on Provided Context**\n...,### Prohibitions Based on Provided Context\n\n...
1,Business RequirementUnited States banking orga...,The business requirements for Regulation K sta...,### Detailed Requirements for Regulation K\n\n...,## **Permissions for United States Banking Org...,## Prohibitions Based on Regulation K\n\n- **L...
2,Business RequirementIf a U.S. banking organiza...,To engage in data processing or data transmiss...,Based on the provided **Verbatim Business Requ...,## **Permissions for U.S. Banking Organization...,### Prohibitions Based on Regulation K\n\n- U....
3,Business RequirementAny banking organization t...,To engage in underwriting securities distribut...,"I'm sorry, but I cannot provide detailed requi...",## **Permissions Based on Provided Information...,## Prohibitions\n\n- Banking organizations are...
4,Business RequirementNo foreign bank may establ...,The business requirement for Regulation K stat...,"I'm sorry, but I cannot provide a detailed lis...",## Permissions Based on Regulation K\n\n- **Es...,### Prohibitions Based on Verbatim Business Re...


In [40]:
full_markdown = " ".join([regulation_summary, table_markdown, key_terms_markdown])
full_markdown = full_markdown.replace("```markdown\n", "\n")
full_markdown = full_markdown.replace("```", "")
display(Markdown(full_markdown))

Regulation K is a set of rules established by the Federal Reserve that governs the international operations of U.S. banking organizations, including the establishment and operation of foreign branches and subsidiaries. Its core purpose is to ensure that these international activities are conducted in a safe and sound manner, while also promoting the competitiveness of U.S. banks in the global market. The regulation outlines the requirements for obtaining approval to establish foreign branches, the permissible activities these branches can engage in, and the capital adequacy standards they must maintain. It also addresses the reporting and record-keeping obligations of U.S. banks with international operations. The impact of Regulation K is significant as it provides a framework for U.S. banks to expand their operations globally while ensuring compliance with U.S. banking laws and regulations. 
|**Business Requirements**|**Simplified Business Requirements**|**Requirements, Permissions, Prohibitions**|
|-----------------------|----------------------------------|-----------------------------------------|
| Business Requirement: Banking institutions shall account for fees on international loans in accordance with generally accepted accounting principles. | The business requirement for Regulation K states that banking institutions must account for fees on international loans following generally accepted accounting principles. This ensures that the financial reporting of these fees is consistent and transparent. | <br> - **Requirement:** <br> - Understand the Regulation <br> - Conduct a gap analysis <br> - Develop a compliance plan <br> - Implement changes <br> - Monitor and audit <br> - Documentation and reporting <br> - Review and update <br> - **Permission:** <br> - Accounting practices must adhere to GAAP. <br> - Review and analyze Regulation K. <br> - Conduct gap analysis. <br> - Develop compliance plans. <br> - Update policies and procedures. <br> - Train staff on compliance. <br> - Establish monitoring systems. <br> - Maintain documentation. <br> - Periodically review compliance measures. <br> - **Prohibition:** <br> - Prohibited from non-compliance with GAAP. |
| Business Requirement: United States banking organizations, with the prior consent of the Board, may acquire and hold investments in foreign companies that do business in the United States subject to the following conditions: (1) the foreign company is engaged predominantly in business outside the United States or in internationally related activities in the United States; (2) the direct or indirect activities of the foreign company in the United States are either banking or closely related to banking; and (3) the United States banking organization does not own 25 percent or more of the voting stock of, or otherwise control, the foreign company. | The business requirements for Regulation K state that U.S. banking organizations, with prior Board consent, can acquire and hold investments in foreign companies that operate in the U.S. under certain conditions. | <br> - **Requirement:** <br> - Obtain prior consent from the Board <br> - Verify business engagement outside the U.S. <br> - Ensure activities are banking-related <br> - Confirm ownership limitations <br> - Compliance monitoring <br> - Documentation and reporting <br> - Legal and regulatory review <br> - Training and awareness <br> - **Permission:** <br> - Permitted to acquire investments with Board consent. <br> - Allowed to invest in foreign companies engaged outside the U.S. <br> - Permitted to hold investments if ownership is below 25%. <br> - **Prohibition:** <br> - Prohibited from acquiring investments without Board consent. <br> - Prohibited from engaging in non-banking activities. <br> - Prohibited from owning 25% or more of voting stock. |
| Business Requirement: If a U.S. banking organization wishes to engage abroad in data processing or data transmission activities beyond those described in Regulation Y, it must apply for the Board's prior consent under Sec. 211.5(d)(20) of Regulation K. | To engage in data processing or data transmission activities abroad that go beyond what is outlined in Regulation Y, a U.S. banking organization must seek prior consent from the Board as specified in Sec. 211.5(d)(20) of Regulation K. | <br> - **Requirement:** <br> - Understand Regulation K and Y <br> - Identify activities beyond Regulation Y <br> - Prepare an application for Board consent <br> - Submit the application <br> - Await Board decision <br> - Implement approved activities <br> - Maintain compliance and reporting <br> - **Permission:** <br> - Permitted to engage in data activities with Board consent. <br> - Allowed to submit applications for activities beyond Regulation Y. <br> - **Prohibition:** <br> - Prohibited from engaging in activities without Board consent. <br> - Prohibited from commencing activities before consent. |
| Business Requirement: Any banking organization that wishes to engage in underwriting of securities to be distributed in the United States must either be a financial holding company under the Gramm-Leach-Bliley Act or have authority to engage in underwriting activity under section 4(c)(8) of the Bank Holding Company Act. | To engage in underwriting securities distributed in the United States, a banking organization must either be a financial holding company as defined by the Gramm-Leach-Bliley Act or possess the authority to conduct underwriting activities under section 4(c)(8) of the Bank Holding Company Act. | <br> - **Requirement:** <br> - Meet criteria for underwriting <br> - Compliance with relevant acts <br> - **Permission:** <br> - Permitted to underwrite if a financial holding company. <br> - Allowed to underwrite with authority under the Bank Holding Company Act. <br> - **Prohibition:** <br> - Prohibited from underwriting without meeting criteria. |
| Business Requirement: No foreign bank may establish a branch or an agency, or acquire ownership or control of a commercial lending company, without the prior approval of the Board. | The business requirement for Regulation K states that no foreign bank is allowed to establish a branch or agency, or acquire ownership or control of a commercial lending company, without obtaining prior approval from the Board. | <br> - **Requirement:** <br> - Obtain prior approval for establishment <br> - Submit a formal application <br> - Compliance with local laws <br> - Financial and operational requirements <br> - Post-approval obligations <br> - Regular reporting <br> - **Permission:** <br> - Permitted to establish branches with Board approval. <br> - Allowed to acquire companies with prior approval. <br> - **Prohibition:** <br> - Prohibited from establishing branches without approval. <br> - Prohibited from acquiring companies without approval. |
 
- **International Banking Operations**: Activities conducted by banks that involve transactions and services across national borders.
- **Foreign Branches**: Offices established by a bank in a country outside its home country to conduct banking activities.
- **Subsidiaries**: Companies that are controlled by a parent bank, often located in a different country, which operate under the laws of that country.
- **Joint Ventures**: Business arrangements where two or more parties collaborate to undertake a specific project or business activity, sharing resources and risks.
- **Capital Requirements**: The minimum amount of capital that banks must hold to ensure they can meet their financial obligations and absorb losses.
- **Reporting Requirements**: Obligations for banks to provide regular financial and operational information to regulatory authorities.
- **Compliance**: Adhering to laws, regulations, and guidelines set forth by governing bodies in the banking sector.
- **Foreign Banks**: Banks that are based outside of the United States but may operate within the U.S. under specific regulations.
- **U.S. Banking Laws**: The set of regulations that govern the operations of banks within the United States, ensuring safety and soundness in the banking system.


In [36]:
full_markdown

"Regulation K governs the international banking operations of U.S. banks and the activities of foreign banks in the United States. Its core purpose is to facilitate the international expansion of U.S. banks while ensuring that their overseas activities are conducted safely and soundly. The regulation outlines the requirements for establishing foreign branches, subsidiaries, and joint ventures, and it sets forth the conditions under which U.S. banks can invest in foreign entities. Additionally, Regulation K provides guidelines for foreign banks operating in the U.S., ensuring they adhere to similar standards as domestic banks. The regulation impacts how banks manage their international operations, requiring them to maintain adequate capital, adhere to reporting requirements, and comply with U.S. banking laws and regulations. \n|**Business Requirements**|**Simplified Business Requirements**|**Requirements, Permissions, Prohibitions**|\n|-----------------------|---------------------------